In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation,LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


In [8]:
path  = get_file('nietzsche.txt' , origin = 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()

In [13]:
len(text)

600901

In [24]:
chars = sorted(list(set(text)))
len(chars)

char_indices = dict((c,i) for i,c in enumerate(chars))
indices_char = dict((i,c) for i,c in enumerate(chars))

#print (char_indices[' '])

1


In [37]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
#print (len(sentences))
#print( next_chars[0])

200287
n


In [38]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
y[i, char_indices[next_chars[i]]] = 1


In [39]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr = 0.01))


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (None, 128)           96256       lstm_input_1[0][0]               
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 59)            7611        lstm_1[0][0]                     
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 59)            0           dense_1[0][0]                    
Total params: 103,867
Trainable params: 103,867
Non-trainable params: 0
____________________________________________________________________________________________________


In [50]:
 hist = model.fit(X, y, batch_size=128,nb_epoch=10)

Epoch 1/10
200287/200287 [==============================] - 430s - loss: 8.9279e-13     

In [51]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [72]:


for diversity in [0.2]:
        print()
        print('----- diversity:', diversity)
        start_index = random.randint(0, len(text) - maxlen - 1)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(4):

            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.
            
            #print (x)

            preds = model.predict(x, verbose=0)[0]
            #print (preds.shape)
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:]+ next_char
            print (sentence)

            #sys.stdout.write(next_char)
            #sys.stdout.flush()    
            #print()


----- diversity: 0.2
----- Generating with seed: " philosophy. (such seems to
me, for inst"
 philosophy. (such seems to
me, for instphilosophy. (such seems to
me, for insts
hilosophy. (such seems to
me, for instss
ilosophy. (such seems to
me, for instsss
losophy. (such seems to
me, for instssss
